<a href="https://colab.research.google.com/github/crazy85128x/ML0930/blob/master/nlp_sentiment_average.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import tensorflow as tf

dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True,
)

In [22]:
import glob
# glob.glob("/root/.keras/datasets/aclImdb/train/pos/*")
import os
import pandas as pd
def read(path):
    with open(path, "r", encoding="utf-8") as f:
        content = f.read()
    return content

def get_data(t):
    dn = os.path.dirname(dataset)
    pattern = os.path.join(dn, "aclImdb", t, "pos", "*.txt")
    pos = glob.glob(pattern)
    pattern = os.path.join(dn, "aclImdb", t, "neg", "*.txt")
    neg = glob.glob(pattern)
    sentiments = [1] * len(pos) + [0] * len(neg)
    contents = map(read, pos + neg)
    df = pd.DataFrame({
        "contents":contents,
        "sentiment":sentiments
    })
    return df

train_df = get_data("train")
test_df = get_data("test")
test_df

,contents,sentiment
0,"It kept me on the edge of my seat. True, the s...",1
1,Born in 1946 I was about eight years old when ...,1
2,In the 60's Cleveland television audiences cou...,1
3,I think it is a brilliant show with cool talki...,1
4,John Wayne's first starring role just blew me ...,1
...,...,...
24995,I am not a big fan of the Spielberg/Cruise ver...,0
24996,The director infuses this film with false dept...,0
24997,"As a flying and war movie buff, this ranks at ...",0
24998,I rented this movie under the impression that ...,0


In [23]:
TOK = 3000
LEN = 512
EM = 128

In [24]:
from tensorflow.keras.preprocessing.text import Tokenizer
tok = Tokenizer(num_words=TOK)
# fit_on_texts: fit_transform 的 fit
tok.fit_on_texts(train_df["contents"])
# seq: 把所有單字換成數字的序列
x_train_seq = tok.texts_to_sequences(train_df["contents"])
x_test_seq = tok.texts_to_sequences(test_df["contents"])

In [25]:
# tok.word_index
tok.index_word[2]

'and'

In [26]:
# pd.DataFrame(train_df_seq)
# truncating: pre 12345 -> 45 post 12345 -> 12
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train_pad = pad_sequences(x_train_seq, LEN)
x_test_pad = pad_sequences(x_test_seq, LEN)
pd.DataFrame(x_test_pad)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,8,3,1161,12,1,17,6,21,1867,14,45,12,68,3,2178,15,49,19,280,54,39,1029,8,11,28,40,3,706,487,178,5,121,86,11,62,627,277,22,377,146
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,13,31,666,2041,1532,1,1437,606,37,3,247,8,58,702,9,13,37,316,32,776,877,9,90,3,524,133,12,73,430,276,54,28,77,166,11,730,18,9,1674,69
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,728,196,1,2,1,1393,1,106,13,1,29,55,115,265,1016,431,53,5,12,55,2,125,71,256,234,1558,6,305,7,7,92,171,48,78,10,121,143,61,3,504
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,4,792,23,134,22,200,64,86,9,492,8,1,1844,162,10,101,11,6,28,4,1,115,245,284,2,10,481,383,11,5,29,346,34,37,369,7,7,15,58,1911
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,97,1,254,248,6,32,1,995,1489,1847,23,2437,18,1,19,14,3,223,6,21,10,188,101,4,157,422,12,67,1556,1,191,8,3,278,4,578,476,20,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,10,822,12,230,2315,50,71,3,644,1755,8,1,624,485,307,35,5,10,479,10,25,5,137,16,1,30,219,9,405,1722,99,3,321,30,1,1710,1064,1214,1688,2335
24996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,169,2752,2,26,57,8,108,47,23,197,131,104,99,1,290,1467,109,12,109,47,1098,364,2,149,20,1464,2,5,984,3,582,134,995,180,72,190,15,2484,8,110
24997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2,12,1,84,81,33,23,266,5,27,162,119,11,91,217,14,45,33,801,5,898,8,14,108,2869,14,611,47,23,108,49,1544,105,2,11,6,21,28,4,95
24998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,299,6,48,163,11,17,287,2739,7,7,1,315,941,1468,3,1234,646,80,1,174,2,1442,1919,4,1419,7,7,511,344,1,4,58,2037,177,5,11,52,558,4,55


In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import Dense, Flatten, Dropout

layers = [
  # 1~3000(token) + 0(padding)
  # params: 3001 * 128
  Embedding(TOK+1, EM, mask_zero=True, input_length=LEN),
  GlobalAveragePooling1D(),
  Dense(2, activation="softmax")  
]
model = Sequential(layers)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 512, 128)          384128    
_________________________________________________________________
global_average_pooling1d_2 ( (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 258       
Total params: 384,386
Trainable params: 384,386
Non-trainable params: 0
_________________________________________________________________


In [28]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
model.compile(loss=SparseCategoricalCrossentropy(),
       optimizer=Adam(),
       metrics=["accuracy"])

In [29]:
y_train = train_df["sentiment"]
y_test = test_df["sentiment"]

In [30]:
# epoch: 整份資料要看幾遍
# 60000筆, 2epoch, 200batch -> 60000 * 2 / 200次調整
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
callbacks = [
    ModelCheckpoint("model.h5", save_best_only=True),
    EarlyStopping(patience=5, restore_best_weights=True)
]
model.fit(x_train_pad, 
     y_train,
     batch_size=100,
     epochs=50,
     validation_split=0.1,
     callbacks=callbacks,
     verbose=2)

Epoch 1/50
225/225 - 4s - loss: 0.5679 - accuracy: 0.7170 - val_loss: 0.4885 - val_accuracy: 0.7828
Epoch 2/50
225/225 - 4s - loss: 0.3606 - accuracy: 0.8649 - val_loss: 0.4322 - val_accuracy: 0.8040
Epoch 3/50
225/225 - 4s - loss: 0.2914 - accuracy: 0.8884 - val_loss: 0.3840 - val_accuracy: 0.8324
Epoch 4/50
225/225 - 4s - loss: 0.2592 - accuracy: 0.8995 - val_loss: 0.3532 - val_accuracy: 0.8512
Epoch 5/50
225/225 - 4s - loss: 0.2410 - accuracy: 0.9069 - val_loss: 0.2999 - val_accuracy: 0.8796
Epoch 6/50
225/225 - 4s - loss: 0.2281 - accuracy: 0.9126 - val_loss: 0.3598 - val_accuracy: 0.8532
Epoch 7/50
225/225 - 4s - loss: 0.2185 - accuracy: 0.9171 - val_loss: 0.3630 - val_accuracy: 0.8516
Epoch 8/50
225/225 - 4s - loss: 0.2122 - accuracy: 0.9208 - val_loss: 0.3651 - val_accuracy: 0.8508
Epoch 9/50
225/225 - 3s - loss: 0.2059 - accuracy: 0.9229 - val_loss: 0.3370 - val_accuracy: 0.8648
Epoch 10/50
225/225 - 3s - loss: 0.2023 - accuracy: 0.9228 - val_loss: 0.3746 - val_accuracy: 0.8504

In [31]:
model.evaluate(x_test_pad, y_test)

782/782 [==============================] - 3s 4ms/step - loss: 0.2866 - accuracy: 0.8833


[0.28656646609306335, 0.8833199739456177]

In [32]:
layers = [
  Embedding(TOK+1, EM, mask_zero=True),
  GlobalAveragePooling1D()
]
w = model.layers[0].get_weights()
infer = Sequential(layers)
infer.layers[0].set_weights(w)
infer.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 128)         384128    
_________________________________________________________________
global_average_pooling1d_3 ( (None, 128)               0         
Total params: 384,128
Trainable params: 384,128
Non-trainable params: 0
_________________________________________________________________


In [33]:
# w
# infer.layers[0].get_weights()
# 25000, 512
# [512list, 512list]
target = "the"
# [1list]
pre = infer.predict([[tok.word_index[target]]])
pre[0]

array([-0.00977593, -0.05324828,  0.02374108, -0.04741118, -0.04979361,
       -0.02772495,  0.02774241, -0.02763193, -0.01296664, -0.02354411,
        0.01891326,  0.06074545,  0.05990081, -0.00808212, -0.0558381 ,
       -0.02235386,  0.0653808 , -0.01772021, -0.0093825 ,  0.01057864,
       -0.01843845, -0.02963819,  0.00426034, -0.02964092, -0.0621849 ,
       -0.03185329,  0.00181961,  0.03332917, -0.01891471, -0.03097513,
        0.02149317,  0.03439151,  0.02186285,  0.02531382,  0.06607596,
       -0.01678765,  0.05957953, -0.03130328,  0.05496573, -0.02261335,
        0.05764224, -0.03657501,  0.01418815, -0.04774017, -0.05623996,
       -0.02335511, -0.00923751,  0.01819639,  0.0243934 ,  0.00316916,
        0.04744363, -0.02178562, -0.01330646,  0.03017424, -0.0358944 ,
        0.05850036,  0.02934222, -0.03369678, -0.02798837,  0.00173662,
       -0.00391671,  0.00517199, -0.01703883,  0.02141908,  0.06036657,
        0.00885221,  0.04595642, -0.00135091, -0.02142078, -0.04

In [34]:
f = open("vec.txt", "w", encoding="utf-8")
f.write("{} {}\n".format(TOK, EM))
for i in range(1, TOK+1):
  #[1list]
  pre = infer.predict([[i]])[0]
  v = " ".join(map(str, pre))
  f.write("{} {}\n".format(tok.index_word[i], v))
f.close

<function TextIOWrapper.close>

In [35]:
from gensim.models import KeyedVectors
w2v = KeyedVectors.load_word2vec_format("vec.txt")

In [36]:
w2v.wv.vocab

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


{'the': <gensim.models.keyedvectors.Vocab at 0x7f044ef6d320>,
 'and': <gensim.models.keyedvectors.Vocab at 0x7f044ef6d4e0>,
 'a': <gensim.models.keyedvectors.Vocab at 0x7f044ef6d3c8>,
 'of': <gensim.models.keyedvectors.Vocab at 0x7f044ef6d2b0>,
 'to': <gensim.models.keyedvectors.Vocab at 0x7f044ef6d198>,
 'is': <gensim.models.keyedvectors.Vocab at 0x7f044ef6d5f8>,
 'br': <gensim.models.keyedvectors.Vocab at 0x7f044ef6d8d0>,
 'in': <gensim.models.keyedvectors.Vocab at 0x7f044ef6d160>,
 'it': <gensim.models.keyedvectors.Vocab at 0x7f044ef6d278>,
 'i': <gensim.models.keyedvectors.Vocab at 0x7f044ef6d908>,
 'this': <gensim.models.keyedvectors.Vocab at 0x7f044ef6d438>,
 'that': <gensim.models.keyedvectors.Vocab at 0x7f04d3447550>,
 'was': <gensim.models.keyedvectors.Vocab at 0x7f04d3447940>,
 'as': <gensim.models.keyedvectors.Vocab at 0x7f04d3447240>,
 'for': <gensim.models.keyedvectors.Vocab at 0x7f04d34472e8>,
 'with': <gensim.models.keyedvectors.Vocab at 0x7f04d3447cf8>,
 'movie': <gensi

In [40]:
w2v.most_similar("me")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('sheriff', 0.678855836391449),
 ('variety', 0.6619827747344971),
 ('lovely', 0.6555631160736084),
 ('michelle', 0.6520454287528992),
 ('now', 0.6518752574920654),
 ('rock', 0.6508750915527344),
 ('easily', 0.6504671573638916),
 ('vs', 0.6486493349075317),
 ('tales', 0.6483433246612549),
 ('originally', 0.6481031775474548)]